# Kickstarter_survivors
### Sifan Liu

## Set up and load data

In [2]:
# Load packages and set up
import numpy as np
import pandas as pd
import json
import glob

def CustomParser(data):
    import json
    try:
        j1 = json.loads(data)
        return j1
    except ValueError:
        pass

JSONconverters = {'location':CustomParser, 'category':CustomParser, 'creator':CustomParser}


In [2]:
# read data

# from csv =============================================
# all files --------------------------------------------
path = r'source/Kickstarter_2018-02-15T03_20_44_743Z'
all_files = glob.glob(path + "/*.csv")

df_from_each_file = (pd.read_csv(f,converters = JSONconverters) for f in all_files)
df = pd.concat(df_from_each_file, ignore_index = True)

In [3]:
# sample_data ------------------------------------------
# df = pd.read_csv('source/kickstarter_sample.csv', converters = JSONconverters)

In [3]:
# extract json information to columns
def JsonConcate(dataframe, column):
    temp = dataframe[column].apply(pd.Series)
    temp = temp.rename(columns = lambda x : column + '_' + str(x))
    dataframe = pd.concat([dataframe[:],temp[:]],axis =1)
    return dataframe

for col in ('location','category','creator'):
    df = JsonConcate(df,col)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195614 entries, 0 to 195613
Data columns (total 62 columns):
backers_count                195614 non-null int64
blurb                        195603 non-null object
category                     195614 non-null object
converted_pledged_amount     195614 non-null int64
country                      195614 non-null object
created_at                   195614 non-null int64
creator                      195614 non-null object
currency                     195614 non-null object
currency_symbol              195614 non-null object
currency_trailing_code       195614 non-null bool
current_currency             195614 non-null object
deadline                     195614 non-null int64
disable_communication        195614 non-null bool
fx_rate                      195614 non-null float64
goal                         195614 non-null float64
id                           195614 non-null int64
is_starrable                 195614 non-null bool
launched_at   

In [10]:
place2msa = pd.read_csv("source/place2msa.csv", encoding = 'latin-1')
# remove 'city'|'town'|'CDP'
place2msa['place'] = place2msa['PLACENAME'].str.replace(r'(\s\w+)$','')

place2msa.sample(5)

,Unnamed: 0,STATE,STATEFP,PLACEFP,PLACENAME,TYPE,FUNCSTAT,cty_name,cty_alt,FIPS_City,...,cbsa_name,metro_micro,csa_name,st_name,countyFIPS,central_outlying,fips,top100,Frey52,place
13543,13544,MI,26,31160,Gaastra city,County Subdivision,F,Iron County,NaN,2631160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaastra
13356,13357,MI,26,23500,Durand city,Incorporated Place,A,Shiawassee County,NaN,2623500,...,"Owosso, MI",Micropolitan Statistical Area,"Lansing-East Lansing-Owosso, MI",Michigan,155.0,Central,26155.0,0.0,0.0,Durand
12520,12521,MA,25,52770,Pepperell CDP,Census Designated Place,S,Middlesex County,NaN,2552770,...,"Boston-Cambridge-Newton, MA-NH",Metropolitan Statistical Area,"Boston-Worcester-Providence, MA-RI-NH-CT",Massachusetts,17.0,Central,25017.0,1.0,1.0,Pepperell
34249,34250,TX,48,100,Abbott city,Incorporated Place,A,Hill County,NaN,4800100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abbott
16484,16485,MN,27,29636,Hollandale city,Incorporated Place,A,Freeborn County,NaN,2729636,...,"Albert Lea, MN",Micropolitan Statistical Area,NaN,Minnesota,47.0,Central,27047.0,0.0,0.0,Hollandale


In [ ]:
## TODO

# merge population data

## Clean Kickstarter dataset

In [4]:
# convert unix time
time_cols = ['created_at', 'deadline', 'state_changed_at','launched_at']
df[time_cols] = df[time_cols].apply(pd.to_datetime,unit='s')

In [5]:
# calculate duration
df['life'] = df['deadline'] - df['launched_at']
df['duration'] = df['state_changed_at'] - df['launched_at']
df['prep'] =  df['launched_at'] - df['created_at']

# df['state_changed_at'][0].year

In [6]:
# factorize project state
df['state'] = df['state'].astype('category')
pd.pivot_table(df, index = ['state'])

,backers_count,category_color,category_id,category_parent_id,category_position,converted_pledged_amount,creator_id,creator_is_registered,currency_trailing_code,disable_communication,fx_rate,goal,id,is_starrable,location_id,pledged,spotlight,staff_pick,static_usd_rate,usd_pledged
state,,,,,,,,,,,,,,,,,,,,
canceled,14.587102,1.127952e+07,251.370169,12.062110,6.537346,1743.798255,1.078425e+09,1.0,0.819336,0.0,1.016327,129306.888268,1.081208e+09,0.000000,3.405093e+06,2071.929268,0.0,0.034115,1.018392,1745.899967
failed,12.971290,1.171623e+07,217.489475,11.868382,6.907461,1139.126898,1.073903e+09,1.0,0.844121,0.0,1.018013,87866.267453,1.077278e+09,0.000000,3.492088e+06,1350.028128,0.0,0.035079,1.023096,1140.815263
live,86.059191,1.030409e+07,143.125364,11.420201,6.517307,6966.336678,1.062947e+09,1.0,0.765998,0.0,0.993470,44983.583915,1.067016e+09,0.939936,3.629417e+06,18937.739379,0.0,0.068063,0.995623,6971.889767
successful,251.701249,1.191508e+07,113.756898,11.720128,7.479788,20586.952318,1.072928e+09,1.0,0.902621,0.0,1.016613,10119.826796,1.073564e+09,0.000000,3.270084e+06,21753.086148,1.0,0.217335,1.020974,20589.072345
suspended,25.078203,9.734057e+06,281.163062,12.978369,5.725458,3818.810316,1.090515e+09,1.0,0.795341,1.0,1.018514,248081.559068,1.107546e+09,0.000000,3.151386e+06,4122.225191,0.0,0.000000,1.018617,3804.140207


In [4]:
df['category_broad'] = df['category_slug'].str.extract(r'([^\/]+)', expand = False).astype('category')
df['category_broad'].describe()

count     222522
unique        15
top        music
freq       30885
Name: category_broad, dtype: object

In [7]:
# generate gender by first name
# STEP 1: strip first name
df['first_name'] = df['creator_name'].str.extract(r'([^\s]+)',expand = False).str.title()

# STEP 2: gender
# https://pypi.org/project/gender-guesser/
import gender_guesser.detector as gender
d = gender.Detector()

# STEP 3: apply
df['gender'] = df['first_name'].apply(d.get_gender)

pd.pivot_table(df, index = ['gender'],aggfunc = len)

,backers_count,blurb,category,category_color,category_id,category_name,category_parent_id,category_position,category_slug,category_urls,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
gender,,,,,,,,,,,,,,,,,,,,,
andy,1997,1997,1997,1997,1997,1997,1997.0,1997,1997,1997,...,1997,1997,1997,1997,1997,1997,1997.0,1997,1997.0,1997
female,35752,35752,35752,35752,35752,35752,35752.0,35752,35752,35752,...,35752,35752,35752,35752,35752,35752,35752.0,35752,35752.0,35752
male,86090,86090,86090,86090,86090,86090,86090.0,86090,86090,86090,...,86090,86090,86090,86090,86090,86090,86090.0,86090,86090.0,86090
mostly_female,7861,7861,7861,7861,7861,7861,7861.0,7861,7861,7861,...,7861,7861,7861,7861,7861,7861,7861.0,7861,7861.0,7861
mostly_male,8303,8303,8303,8303,8303,8303,8303.0,8303,8303,8303,...,8303,8303,8303,8303,8303,8303,8303.0,8303,8303.0,8303
unknown,55611,55611,55611,55611,55611,55611,55611.0,55611,55611,55611,...,55611,55611,55611,55611,55611,55611,55611.0,55611,55611.0,55611


In [10]:
# match place to MSA

df = pd.merge(df,place2msa,how = 'left',
              left_on=['location_state','location_localized_name'],
              right_on=['STATE','place'])


## Summary: selected columns

In [8]:
# keep selected columns
col = ['location_country','location_state','location_localized_name','cbsa_name','top100',
       'backers_count','goal','pledged','creator_name','gender','category_broad','category_name',
       'launched_at','deadline','state_changed_at','state','life','duration','prep']

df_sample = df[col]

# table = pd.pivot_table(df_sample, index=["location_state","location_country"], aggfunc = [np.mean,len])
# table.query('location_country == ["US"]')

In [3]:
# df = pd.read_pickle("master")

In [9]:
# df_sample.to_pickle("sample")
df_sample.to_pickle("master")